## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import os
from os import getcwd
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Taltal,CL,-25.4000,-70.4833,60.98,71,100,0.96,overcast clouds
1,1,Desbiens,CA,48.4178,-71.9495,83.05,70,100,14.38,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,47.48,93,100,2.30,light rain
3,3,Warri,NG,5.5167,5.7500,74.08,96,100,1.95,overcast clouds
4,4,Hobart,AU,-42.8794,147.3294,47.26,74,40,9.22,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
708,708,Wiwili,NI,13.6268,-85.8254,74.98,83,100,4.34,overcast clouds
709,709,Batemans Bay,AU,-35.7167,150.1833,54.10,47,2,10.27,clear sky
710,710,Rypefjord,NO,70.6333,23.6667,61.27,51,0,2.30,clear sky
711,711,Starogard Gdanski,PL,53.9640,18.5264,65.10,100,0,4.99,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
clean_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Desbiens,CA,48.4178,-71.9495,83.05,70,100,14.38,overcast clouds
7,7,Dalbandin,PK,28.8947,64.4101,86.32,37,2,4.56,clear sky
11,11,Albany,US,42.6001,-73.9662,83.05,66,24,1.01,few clouds
13,13,Port Blair,IN,11.6667,92.7500,77.04,100,40,0.00,mist
15,15,Faanui,PF,-16.4833,-151.7500,79.70,75,9,17.49,light rain
17,17,Fare,PF,-16.7000,-151.0167,80.13,74,22,18.84,light rain
19,19,Jamestown,US,42.0970,-79.2353,76.59,91,100,5.75,overcast clouds
24,24,Bethel,US,41.3712,-73.4140,78.19,59,60,9.42,broken clouds
26,26,Guerrero Negro,MX,27.9769,-114.0611,79.63,54,17,14.27,few clouds
32,32,Quchan,IR,37.1060,58.5095,75.74,27,0,6.82,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
clean_df.count()


City_ID                230
City                   230
Country                230
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Current Description    230
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Desbiens,CA,83.05,overcast clouds,48.4178,-71.9495,
7,Dalbandin,PK,86.32,clear sky,28.8947,64.4101,
11,Albany,US,83.05,few clouds,42.6001,-73.9662,
13,Port Blair,IN,77.04,mist,11.6667,92.7500,
15,Faanui,PF,79.70,light rain,-16.4833,-151.7500,
17,Fare,PF,80.13,light rain,-16.7000,-151.0167,
19,Jamestown,US,76.59,overcast clouds,42.0970,-79.2353,
24,Bethel,US,78.19,broken clouds,41.3712,-73.4140,
26,Guerrero Negro,MX,79.63,few clouds,27.9769,-114.0611,
32,Quchan,IR,75.74,clear sky,37.1060,58.5095,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat}, {lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        pass
        
        

Motel Lac St-Jean Inc
Sanjrani House
Welcomhotel By ITC Hotels, Bay Island, Port Blair
Bora-Bora Pearl Beach Resort & Spa
Maitai Lapita Village Huahine
Hampton Inn & Suites Jamestown
Courtyard by Marriott Danbury
Casa Laguna, Bed & Breakfast
گلیم دستبافت کیمیایی
Keleti Beach Resort
Niyama Private Islands Maldives
Green leaf
InterContinental Riyadh, an IHG Hotel
Lovers Point Inn
Halane Hotel and Restaurent
Wood River Inn & Suites
Beach Rotana
Jinjiang Inn Zhoushan Putuo Shenjiamen
Southern Beach Hotel & Resort Okinawa
Mini Central
Hôtel Le Dronmi
Copper Point Resort
Westmark Fairbanks Hotel & Conference Center
Bungalow 63
Nebitchi Hotel
Sheraton Kauai Coconut Beach Resort
Aklavik Inn
LOTTE HOTEL SEOUL
Hilo Hawaiian Hotel
Holiday Inn Express & Suites North Platte, an IHG Hotel
Hôtel Le Lagon
Isles Sunset Lodge
Palais du chef Canton Salamt
Planta Hotel & Residences
Hotel Bon Samaritian
Paradise Inn
Paraiso Escondido
Pondok Wisata Elim
Guorui Hotel
Hotel Jin Hao
Hotel Mustika
INN NEW HORIZ

In [21]:
# 7. Drop the rows where there is no Hotel Name
hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == "NaN"].index, inplace = True)






In [22]:
# Display Hotel Name column
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Desbiens,CA,83.05,overcast clouds,48.4178,-71.9495,Motel Lac St-Jean Inc
7,Dalbandin,PK,86.32,clear sky,28.8947,64.4101,Sanjrani House
13,Port Blair,IN,77.04,mist,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
15,Faanui,PF,79.70,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Fare,PF,80.13,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine
19,Jamestown,US,76.59,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
24,Bethel,US,78.19,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
26,Guerrero Negro,MX,79.63,few clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
32,Quchan,IR,75.74,clear sky,37.1060,58.5095,گلیم دستبافت کیمیایی
33,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_search/WeatherPy_vacation_B.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 


locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))